# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on August 3 (with 2-hour grace period) and worth 50 points

### Overview

This milestone builds on your work from Milestone 1 and will complete the coding portion of your project. You will:

1. Pick 3 modeling algorithms from those we have studied.
2. Evaluate baseline models using default settings.
3. Engineer new features and re-evaluate models.
4. Use feature selection techniques and re-evaluate.
5. Fine-tune for optimal performance.
6. Select your best model and report on your results. 

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 2

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



### Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling. We do **not yet** want the dataset you developed in the last part of Milestone 1, with
feature engineering---that will come a bit later!

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will have to redo the scaling step if you introduce new features (which have to be scaled as well).


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
# Step 1: Load the cleaned dataset
df = pd.read_csv("zillow_cleaned.csv")
# Step 2: Separate features (X) and target (y)
y = df["taxvaluedollarcnt"]
X = df.drop(columns=["taxvaluedollarcnt"])
# Encode categorical variables
X = pd.get_dummies(X, drop_first=True)
# Step 4: Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=2
)
# Impute missing values with mean
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

### Part 1: Picking Three Models and Establishing Baselines [6 pts]

Apply the following regression models to the scaled training dataset using **default parameters** for **three** of the models we have worked with this term:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each of the three models:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV MAE Score**. 


In [3]:
# Add as many cells as you need
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, make_scorer

# Define repeated K-Fold and MAE scorer
cv = RepeatedKFold(n_splits=2, n_repeats=2, random_state=2)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Store results
results = {}

# 1. Linear Regression
model_lr = LinearRegression()
scores_lr = cross_val_score(model_lr, X_train_scaled, y_train, cv=cv, scoring=mae_scorer)
results['Linear Regression'] = (-scores_lr.mean(), scores_lr.std())

# 2. Decision Tree
model_dt = DecisionTreeRegressor(random_state=3)
scores_dt = cross_val_score(model_dt, X_train_scaled, y_train, cv=cv, scoring=mae_scorer)
results['Decision Tree'] = (-scores_dt.mean(), scores_dt.std())

# 3. Random Forest
model_rf = RandomForestRegressor(random_state=3)
scores_rf = cross_val_score(model_rf, X_train_scaled, y_train, cv=cv, scoring=mae_scorer)
results['Random Forest'] = (-scores_rf.mean(), scores_rf.std())

# Print results
for model, (mean_mae, std_mae) in results.items():
    print(f"{model}: Mean MAE = {mean_mae:.2f}, Std = {std_mae:.2f}")


: 

### Part 1: Discussion [3 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which model performed best overall?
  - Which was most stable (lowest std)?
  - Any signs of overfitting or underfitting?

Best Performance:
The Random Forest model worked best, with the lowest average error (about 185,265), meaning its predictions were most accurate.

Most Stable:
The Decision Tree was the most consistent, with the smallest changes in error across tests.

Overfitting/Underfitting:

Linear Regression did very badly, likely due to problems with the data or not fitting well (underfitting).

Decision Tree might be overfitting because it's more flexible and less general.

Random Forest gave a good balance of accuracy and general results.

### Part 2: Feature Engineering [6 pts]

Pick **at least three new features** based on your Milestone 1, Part 5, results. You may pick new ones or
use the same ones you chose for Milestone 1. 

Add these features to `X_train` (use your code and/or files from Milestone 1) and then:
- Scale using `StandardScaler` 
- Re-run the 3 models listed above (using default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
# Add as many cells as you need
# Feature Engineering
def create_features(df):
    df_fe = df.copy()
    # 1. Property age (current year - year built)
    df_fe['property_age'] = 2023 - df_fe['yearbuilt']
    # 2. Room density (bedrooms per square foot)
    df_fe['bedroom_density'] = df_fe['bedroomcnt'] / (df_fe['calculatedfinishedsquarefeet'] + 1)  
    # 3. Luxury ratio (bathrooms per bedroom)
    df_fe['luxury_ratio'] = df_fe['bathroomcnt'] / (df_fe['bedroomcnt'] + 0.01)  
    return df_fe
# Apply feature engineering to training and test sets
X_train_fe = create_features(X_train)
X_test_fe = create_features(X_test)
# Scale features (including new ones)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_fe)
X_test_scaled = scaler.transform(X_test_fe)
# Define models to test
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=3),
    'Random Forest': RandomForestRegressor(random_state=3)
}
# Evaluation setup
cv = RepeatedKFold(n_splits=2, n_repeats=2, random_state=3)
results = {}
# Evaluate each model
for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, 
                           cv=cv, 
                           scoring=make_scorer(mean_absolute_error, greater_is_better=False))
    
    # Store mean and std of MAE (convert back to positive values)
    results[name] = (-scores.mean(), scores.std())
    # Print immediate results
    print(f"{name}:")
    print(f"  Mean MAE = ${-scores.mean():,.2f}")
    print(f"  Std MAE = ±${scores.std():,.2f}")
    print(f"  Range = ${-scores.min():,.2f} to ${-scores.max():,.2f}\n")
# Compare with original results
print("\nImprovement from Feature Engineering:")
for model in results:
    improvement = (results[model][0] - original_results[model][0]) / original_results[model][0] * 100
    print(f"{model}: {improvement:.1f}% MAE reduction")

NameError: name 'X_train' is not defined

### Part 2: Discussion [3 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?




> Your text here

### Part 3: Feature Selection [6 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features (with default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
# Add as many cells as you need
from sklearn.feature_selection import SelectFromModel

# 1. Get important features from Random Forest (easiest method)
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_scaled, y_train)

# Select features with importance > average
selector = SelectFromModel(rf, threshold='mean')
selector.fit(X_train_scaled, y_train)

# Get selected features
selected_features = X_train.columns[selector.get_support()]
print(f"Selected {len(selected_features)} important features:")
print(selected_features)

# 2. Use only these features for all models
X_train_selected = X_train_scaled[:, selector.get_support()]

# 3. Re-evaluate models with selected features
print("\nModel Performance with Selected Features:")
for name, model in models.items():
    scores = cross_val_score(model, X_train_selected, y_train,
                           cv=cv,
                           scoring='neg_mean_absolute_error')
    
    mae = -scores.mean()
    std = scores.std()
    print(f"{name}:")
    print(f"  MAE = ${mae:,.0f} (±${std:,.0f})")
    print(f"  Using {len(selected_features)}/{X_train.shape[1]} features\n")

### Part 3: Discussion [3 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?


> Your text here

### Part 4: Fine-Tuning Your Three Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far, and pick a winner!

1. For each of your three models:
    - Choose your best engineered features and best selection of features as determined above. 
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, `Optuna`, etc. as you have practiced in previous homeworks. 
3. Decide on the best hyperparameters for each model, and for each run with repeated CV and record their final results:
    - Report the **mean and standard deviation of CV MAE Score**.  

In [ ]:
# Add as many cells as you need
from sklearn.model_selection import GridSearchCV

# 1. Define the best feature set (from Part 3)
X_train_final = X_train_scaled[:, selector.get_support()]  # Using selected features

# 2. Set up parameter grids for each model
param_grids = {
    'Linear Regression': {
        'fit_intercept': [True, False],
        'positive': [True, False]
    },
    'Random Forest': {
        'n_estimators': [50, 100],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5]
    }
}

# 3. Tune and evaluate each model
final_results = {}

for name, model in models.items():
    print(f"\nTuning {name}...")
    
    # Setup grid search
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=1
    )
    
    # Run tuning
    gs.fit(X_train_final, y_train)
    
    # Store best results
    final_results[name] = {
        'best_params': gs.best_params_,
        'best_score': -gs.best_score_,
        'best_std': gs.cv_results_['std_test_score'][gs.best_index_]
    }
    
    print(f"Best {name} params: {gs.best_params_}")
    print(f"Best MAE: ${-gs.best_score_:,.0f} (±${gs.cv_results_['std_test_score'][gs.best_index_]:,.0f})")

# 4. Compare final results
print("\nFinal Model Comparison:")
for name, res in final_results.items():
    print(f"{name}:")
    print(f"  MAE = ${res['best_score']:,.0f} (±${res['best_std']:,.0f})")
    print(f"  Params: {res['best_params']}\n")

### Part 4: Discussion [3 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?


> Your text here

### Part 5: Final Model and Design Reassessment [6 pts]

In this part, you will finalize your best-performing model.  You’ll also consolidate and present the key code used to run your model on the preprocessed dataset.
**Requirements:**

- Decide one your final model among the three contestants. 

- Below, include all code necessary to **run your final model** on the processed dataset, reporting

    - Mean and standard deviation of CV MAE Score.
    
    - Test score on held-out test set. 

In [ ]:
# Add as many cells as you need



### Part 5: Discussion [8 pts]

In this final step, your goal is to synthesize your entire modeling process and assess how your earlier decisions influenced the outcome. Please address the following:

1. Model Selection:
- Clearly state which model you selected as your final model and why.

- What metrics or observations led you to this decision?

- Were there trade-offs (e.g., interpretability vs. performance) that influenced your choice?

2. Revisiting an Early Decision

- Identify one specific preprocessing or feature engineering decision from Milestone 1 (e.g., how you handled missing values, how you scaled or encoded a variable, or whether you created interaction or polynomial terms).

- Explain the rationale for that decision at the time: What were you hoping it would achieve?

- Now that you've seen the full modeling pipeline and final results, reflect on whether this step helped or hindered performance. Did you keep it, modify it, or remove it?

- Justify your final decision with evidence—such as validation scores, visualizations, or model diagnostics.

3. Lessons Learned

- What insights did you gain about your dataset or your modeling process through this end-to-end workflow?

- If you had more time or data, what would you explore next?

> Your text here